In [1]:
%matplotlib inline

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device", device)

Using device cuda:0


In [2]:
X_dim = 6
c_dim = 21
gridSize = 100
z_dim = 5

bs = 256

In [48]:
from data.AttentionDataset import AttentionDataset

train_loader = DataLoader(AttentionDataset(np_file_data = 'data/NarrowPassage/narrowDataOcc100.npz',
                            sample_dim = X_dim, 
                            condition_dim = c_dim,
                            gridSize = gridSize,
                            train = True),
                         batch_size = bs, shuffle=True)
test_loader = DataLoader(AttentionDataset(np_file_data = 'data/NarrowPassage/narrowDataOcc100.npz',
                            sample_dim = X_dim, 
                            condition_dim = c_dim,
                            gridSize = gridSize,
                            train = False),
                          batch_size = bs, shuffle=True)

In [4]:
import torch.nn.functional as F
class convVAE(nn.Module):
    def __init__(self, sample_size, grid_size, cnnout_size, encoder_layer_sizes, latent_size, decoder_layer_sizes):
        super(convVAE, self).__init__()

        assert type(encoder_layer_sizes) == list
        assert type(latent_size) == int
        assert type(decoder_layer_sizes) == list
        
        self.latent_size = latent_size
        self.condnn = CondNN(sample_size, grid_size, cnnout_size)
        self.encoder = Encoder(sample_size * 3 + cnnout_size, encoder_layer_sizes, latent_size)
        self.decoder = Decoder(latent_size + sample_size * 2 + cnnout_size, decoder_layer_sizes, sample_size)

    def cnn(self,startend, occ):
        return self.cnn(startend, occ)
        
    def encode(self, x):
        return self.encoder(x)

    def decode(self, x):
        return self.decoder(x)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x, startend, occ):
        c = self.condnn(startend, occ)
        mu, logvar = self.encode(torch.cat((x, c), dim=-1))
        z = self.reparameterize(mu, logvar)
        return self.decode(torch.cat((z, c), dim=-1)), mu, logvar
    
    def inference(self, startend, occ, num_viz):
        c = self.condnn(startend, occ)
        z = torch.randn(num_viz, self.latent_size, device = c.device)
        return self.decode(torch.cat((z, c), dim=-1))
    
class Encoder(nn.Module):
    def __init__(self, input_size, layer_sizes, latent_size):
        super(Encoder, self).__init__()

        layer_sizes = [input_size] + layer_sizes
        modules = []
        for i, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            modules.append(nn.Linear(in_size, out_size))
            modules.append(nn.ReLU())
#             modules.append(nn.Dropout(p=0.5))

        self.sequential = nn.Sequential(*modules)
        self.linear_means = nn.Linear(layer_sizes[-1], latent_size)
        self.linear_log_var = nn.Linear(layer_sizes[-1], latent_size)

    def forward(self, x):
        x = self.sequential(x)
        means = self.linear_means(x)
        log_vars = self.linear_log_var(x)
        return means, log_vars


class Decoder(nn.Module):
    def __init__(self, input_size, layer_sizes, sample_size):
        super(Decoder, self).__init__()

        layer_sizes = [input_size] + layer_sizes
        modules = []
        for i, (in_size, out_size) in enumerate(zip(layer_sizes[:-1], layer_sizes[1:])):
            modules.append(nn.Linear(in_size, out_size))
            modules.append(nn.ReLU())
#             modules.append(nn.Dropout(p=0.5))
        modules.append(nn.Linear(layer_sizes[-1], sample_size))

        self.sequential = nn.Sequential(*modules)

    def forward(self, x):
        return self.sequential(x)


class CondNN(nn.Module):
    def __init__(self, sampleSize, gridSize, outSize):
        super(CondNN, self).__init__()
        self.sampleSize = sampleSize
        self.gridSize = gridSize
        self.cnn = nn.Sequential(
                    nn.Conv2d(1, 6, 5, padding=(2,2)),
                    nn.MaxPool2d(3,3),
                    nn.Conv2d(6, 16, 5, padding=(2,2)),
                    nn.MaxPool2d(2,2),
                    nn.Conv2d(16, 48, 5, padding=(2,2)),
                    nn.AdaptiveAvgPool2d((11, 11)))
        self.fc1 = nn.Linear(48 * 11 * 11, 512)
        self.fc2 = nn.Linear(512, outSize)

    def forward(self, startend, occ):
        occ = self.cnn(occ)
        occ = occ.view(-1, 48 * 11 * 11)
        occ = F.relu(self.fc1(occ))
        occ = F.relu(self.fc2(occ))
        x = torch.cat((occ, startend), dim=-1)
#         x = F.relu(self.fc2(x))
        return x


In [5]:
model = convVAE(sample_size = X_dim, 
                  grid_size = gridSize, 
                  cnnout_size = 9,
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
print(model)

convVAE(
  (condnn): CondNN(
    (cnn): Sequential(
      (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
      (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(16, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (5): AdaptiveAvgPool2d(output_size=(11, 11))
    )
    (fc1): Linear(in_features=5808, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=9, bias=True)
  )
  (encoder): Encoder(
    (sequential): Sequential(
      (0): Linear(in_features=27, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=1024, bias=True)
      (3): ReLU()
      (4): Linear(in_features=1024, out_features=512, bias=True)
      (5): ReLU()
    )
    (linear_means): Linear(in_features=512, out_feature

In [6]:
def loss_fn(recon_x, x, w, mean, log_var):
    MSE = torch.mean((w.expand_as(x) * (recon_x-x)**2))
    KLD = - 0.002 * torch.mean(torch.sum(1 + log_var - mean.pow(2) - log_var.exp(), 1))
    return MSE + KLD, MSE

optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [7]:
def train(epoch):
    model.train()
    train_loss = 0
    mse_loss = 0
    w = torch.tensor([1, 1, 1, 0.5, 0.5, 0.5], dtype=torch.float).to(device)
    for batch_idx, (sample, _, startend, occ) in enumerate(train_loader):
        
        sample, startend, occ = sample.to(device), startend.to(device), occ.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(sample, startend, occ)
        loss, mse= loss_fn(recon_batch, sample, w, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        mse_loss += mse.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(sample), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item()))

    epoch_loss = train_loss * len(sample) / len(train_loader.dataset)
    epoch_mse = mse_loss * len(sample) / len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.7f}'.format(
          epoch, epoch_loss))
    return epoch, epoch_loss, epoch_mse

def test(epoch):
    model.eval()
    test_loss = 0
    mse_loss = 0
    w = torch.tensor([1, 1, 1, 0.5, 0.5, 0.5], dtype=torch.float).to(device)
    for batch_idx, (sample, _, startend, occ) in enumerate(test_loader):
        
        sample, startend, occ = sample.to(device), startend.to(device), occ.to(device)
        recon_batch, mu, logvar = model(sample, startend, occ)
        loss, mse= loss_fn(recon_batch, sample, w, mu, logvar)
        test_loss += loss.item()
        mse_loss += mse.item()

    epoch_loss = test_loss * len(sample) / len(test_loader.dataset)
    epoch_mse = mse_loss * len(sample) / len(test_loader.dataset)
    print('====> Epoch: {} Average test loss: {:.7f}'.format(
          epoch, epoch_loss))
    return epoch, epoch_loss, epoch_mse

In [8]:
epoch = 0
# shell python -m visdom.server
from visdom import Visdom
vis = Visdom(env='cnn_0.002_100*100')

loss_window = vis.line(
    Y=torch.zeros((1)).cpu(),
    X=torch.zeros((1)).cpu(),
    opts=dict(xlabel='epoch',ylabel='Loss',title='training loss',legend=['loss']))

In [9]:
for epoch in range(epoch, epoch + 200):
    log_interval = 100
    epoch, epoch_loss, epoch_mse = train(epoch)
    vis.line(X=torch.ones((1,1)).cpu()*epoch,Y=torch.Tensor([epoch_loss]).unsqueeze(0).cpu(),win=loss_window,update='append',name='loss')
    vis.line(X=torch.ones((1,1)).cpu()*epoch,Y=torch.Tensor([epoch_mse]).unsqueeze(0).cpu(),win=loss_window,update='append',name='mse_loss')
    

Train Epoch: 0 [0/66984 (0%)]	Loss: 0.243439
Train Epoch: 0 [25600/66984 (38%)]	Loss: 0.036980
Train Epoch: 0 [51200/66984 (76%)]	Loss: 0.031420
====> Epoch: 0 Average loss: 0.0282066
Train Epoch: 1 [0/66984 (0%)]	Loss: 0.029956
Train Epoch: 1 [25600/66984 (38%)]	Loss: 0.027781
Train Epoch: 1 [51200/66984 (76%)]	Loss: 0.027566
====> Epoch: 1 Average loss: 0.0185148
Train Epoch: 2 [0/66984 (0%)]	Loss: 0.026860
Train Epoch: 2 [25600/66984 (38%)]	Loss: 0.025321
Train Epoch: 2 [51200/66984 (76%)]	Loss: 0.026131
====> Epoch: 2 Average loss: 0.0171411
Train Epoch: 3 [0/66984 (0%)]	Loss: 0.025443
Train Epoch: 3 [25600/66984 (38%)]	Loss: 0.024906
Train Epoch: 3 [51200/66984 (76%)]	Loss: 0.024216
====> Epoch: 3 Average loss: 0.0164511
Train Epoch: 4 [0/66984 (0%)]	Loss: 0.024705
Train Epoch: 4 [25600/66984 (38%)]	Loss: 0.024058
Train Epoch: 4 [51200/66984 (76%)]	Loss: 0.024319
====> Epoch: 4 Average loss: 0.0160205
Train Epoch: 5 [0/66984 (0%)]	Loss: 0.023544
Train Epoch: 5 [25600/66984 (38%)]	

KeyboardInterrupt: 

In [10]:
torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, 'checkpoints/cnn100.pt')

/home/rong/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type convVAE. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/rong/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type CondNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/rong/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Encoder. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/rong/anaconda3/envs/torch/lib/python3.7/site-packages/torch/serialization.py:256: UserWarning: Couldn't retrieve source code for container of type Decoder. It won't be checked for correctness upon loadin

In [11]:
torch.cuda.empty_cache()

model = convVAE(sample_size = X_dim, 
                  grid_size = gridSize, 
                  cnnout_size = 9,
                  encoder_layer_sizes = [512,1024,512], 
                  latent_size = z_dim, 
                  decoder_layer_sizes = [512,1024,512]).to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']

model.eval()

convVAE(
  (condnn): CondNN(
    (cnn): Sequential(
      (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (1): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
      (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (4): Conv2d(16, 48, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (5): AdaptiveAvgPool2d(output_size=(11, 11))
    )
    (fc1): Linear(in_features=5808, out_features=512, bias=True)
    (fc2): Linear(in_features=512, out_features=9, bias=True)
  )
  (encoder): Encoder(
    (sequential): Sequential(
      (0): Linear(in_features=27, out_features=512, bias=True)
      (1): ReLU()
      (2): Linear(in_features=512, out_features=1024, bias=True)
      (3): ReLU()
      (4): Linear(in_features=1024, out_features=512, bias=True)
      (5): ReLU()
    )
    (linear_means): Linear(in_features=512, out_feature

In [ ]:
test_data = test_loader.dataset
viz_idx =   torch.randint(0,len(test_data),[1]).item()  

# viz_idx = 8712

_, con, startend, occ = test_data[viz_idx]
startend, occ = torch.tensor(startend), torch.unsqueeze(torch.tensor(occ), 0)
model.eval()
num_viz = 1000
y_viz = model.inference(startend.expand(num_viz, -1).to(device), 
                        occ.expand(num_viz, -1, -1, -1).to(device), num_viz)

y_viz=y_viz.cpu().detach().numpy()
occ=occ.cpu().detach().numpy()

from utils.NarrowPassage import plotCondition, plotSample, plotSpeed

# plotCondition(condition)
plotSample(y_viz, con)
plotSpeed(y_viz, con)